In [1]:
!tree ../dataset/

../dataset/
├── en_espanol
│   ├── docx2txt.py
│   ├── Второй_жанр_исходная.txt
│   └── Первый_жанр_исходная.txt
├── Второй_жанр_исходная.txt
├── Первый_жанр_исходная.txt
├── Сокращение по частям речи
│   ├── 1.Первый жанр исходная выборка.txt
│   ├── 2.Первый жанр без клауз, включающих наречия.txt
│   ├── 3.Первый жанр без клауз, включающих глаголы.txt
│   ├── 4. Первый жанр без клауз, включающих глаголы и наречия.txt
│   ├── Без прилагательных второй жанр.txt
│   ├── Без прилагательных первый жанр.txt
│   └── Случайные выборки.txt
└── сокращение по частотности
    ├── 1а_ без сокращений.txt
    ├── 1б_Изъяты лексемы с частотой выше 100.txt
    ├── 1в_Изъяты лексемы с частотой выше 49.txt
    ├── 1г_Изъяты лексемы с частотой выше 29.txt
    ├── 1д_Изъяты лексемы с частотой выше 9.txt
    ├── 2а_ без сокращений.txt
    ├── 2б _ Изъяты лексемы с частотой выше 100..txt
    ├── 2в_ Изъяты лексемы с частотой выше 49..txt
    ├── 2г_  Изъяты лексемы с частотой выше 29.txt
    └── 2д_ Изъяты

In [2]:
!cat ../dataset/'Сокращение по частям речи'/'2.Первый жанр без клауз, включающих наречия.txt'

[] Кустарник Ветви темно-коричневые. Годовалые побеги утолщенные, короткие, темно-бурые, при сушке чернеющие, серовато-шерстисто-опушенные. Почки мономорфные, яйцевидные или округло-яйцевидные, по бокам килеватые, темно- или красновато-коричневые, шерстисто-опушенные. Прилистники полусердцевидные, по краю железистопильчатые, голые или опушенные. Черешки желобчатые, буроватые. Листья овальные, продолговатоовальные или продолговато-обратноланцетные, острые или короткозаостренные, в основании ширококлиновидные или округлые, пильчатые, с расставленными мелкими согнутыми зубцами, плотные (жестковатые в сухом состоянии). Генеративные побеги с 2-6 мелкими листьями; оси соцветий густо-волосисто-опушенные. Прицветники обратнояйцевидные или продолговатые, на верхушке тупые или закругленные, бурые, с обеих сторон опушенные длинными волосками. Мужские сережки. Нектарники парные: адаксиальный двулопастный, абаксиальный цельный. Тычинки со свободными, волосисто опушенными нитями. Пыльники овальные, 

In [3]:
import os
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt')
nltk.download('punkt_tab') # Descarga los datos para la correcta tokenización de los idiomas.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
import re
from nltk.tokenize import word_tokenize, sent_tokenize

def segment_text_with_ngrams(text, n=3):  
    """Segmenta el texto dividiéndolo en frases, evitando dividir dentro de paréntesis."""
    
    def replace_inside_parentheses(match):
        """Reemplaza los signos de puntuación dentro de los paréntesis para evitar segmentación incorrecta."""
        content = match.group(0)
        content = content.replace(".", "§").replace("!", "§").replace("?", "§")
        return content  # Devuelve el contenido modificado
    
    text = re.sub(r'\([^()]*\)', replace_inside_parentheses, text)  # Evita problemas con paréntesis anidados

    sentences = sent_tokenize(text, language="russian")  # Segmentar en frases correctamente

    segmented_sentences = []
    for sentence in sentences:
        sentence = sentence.replace("§", ".")  # Restaurar puntuación dentro de paréntesis
        words = word_tokenize(sentence, language="russian")  # Tokenización de palabras
        if words:
            segmented_sentences.append(" ".join(words))

    return [s.strip() for s in segmented_sentences if s.strip()]

def clean_text(text):
    """Elimina texto entre corchetes y caracteres innecesarios."""
    cleaned_text = re.sub(r'\[.*?\]', '', text)  # Eliminar contenido entre corchetes []
    return cleaned_text.strip()

def load_texts(filename, label):
    with open(filename, "r", encoding="utf-8") as f:
        text = f.read()
    
    sentences = segment_text_with_ngrams(text)  
    cleaned_sentences = [clean_text(sentence) for sentence in sentences]
    
    return [{"text": sentence, "label": label} for sentence in cleaned_sentences if sentence]

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

#cargar los datos
genre1 = load_texts("../dataset/Сокращение по частям речи/2.Первый жанр без клауз, включающих наречия.txt", label="0")


#original_gen_1 = load_texts("../dataset/Первый_жанр_исходная.txt.txt", label="0")

all_texts_df = pd.DataFrame(genre1)



# Eliminar filas con valores NaN o vacíos en la columna 'text'
all_texts_df = all_texts_df[all_texts_df['text'].notna()]
all_texts_df = all_texts_df[all_texts_df['text'] != '']

#original_gen_1_df = original_gen_1_df[original_gen_1_df['text'].notna()]
#original_gen_1_df = original_gen_1_df[original_gen_1_df['text'] != '']

# Dividir en train, validation y test
train_df, test_df = train_test_split(all_texts_df, test_size=0.2, random_state=42, stratify=all_texts_df['label'])

# Guardar los conjuntos limpios
all_texts_df.to_csv("data_without_clauses_including_adverbs.csv",index=False)
train_df.to_csv("train_without_clauses_including_adverbs.csv", index=False)
test_df.to_csv("test_without_clauses_including_adverbs.csv", index=False)

#test_df.to_csv("gen_1_orignal.csv", index=False)


print(f"Tamaño del conjunto completo: {len(all_texts_df)}")
print(f"Tamaño del conjunto de entrenamiento: {len(train_df)}")
print(f"Tamaño del conjunto de prueba: {len(test_df)}")

#print(f"Tamaño del conjunto del conjutto gen_1 origina: {len(original_gen_1_df)}")


Tamaño del conjunto completo: 570
Tamaño del conjunto de entrenamiento: 456
Tamaño del conjunto de prueba: 114


In [6]:
#train
train_df = pd.read_csv('train_without_clauses_including_adverbs.csv')
#set original gen1 gen2
test_df = pd.read_csv('../Step_1/data.csv')

In [7]:
from nltk.util import ngrams
from nltk import word_tokenize

# Función para generar ngramas
def generate_ngrams(text, n=2):
    tokens = word_tokenize(text.lower())  # Convertir el texto a minúsculas y tokenizar
    return set(ngrams(tokens, n))  # Generar ngramas y devolverlos como un conjunto

# Comparar ngramas entre conjuntos
def find_similar_samples(train_texts, test_texts, n=3):
    similar_indices = []
    # Generar ngramas para train y val
    train_ngrams = [generate_ngrams(text, n) for text in train_texts]    
    # Comparar con los textos de test
    for i, test_text in enumerate(test_texts):
        test_ngrams = generate_ngrams(test_text, n)
        
        for train_ngram in train_ngrams:
            if test_ngrams & train_ngram:  # Verificar si hay intersección
                similar_indices.append(i)
                break
    
    return similar_indices

In [8]:
train_texts = train_df['text'].astype(str).tolist()
test_texts = test_df['text'].astype(str).tolist()

similar_indices = find_similar_samples(train_texts, test_texts, n=3)
test_df = test_df.drop(similar_indices)


In [9]:
#train_df.to_csv("train_without_adjectives.csv", index=False)
#val_df.to_csv("val_without_adjectives.csv", index=False)
test_df.to_csv("test_orginal.csv", index=False)

In [10]:
print(f"Tamaño del conjunto de entrenamiento: {len(train_df)}")
print(f"Tamaño del conjunto de prueba(original): {len(test_df)}")

Tamaño del conjunto de entrenamiento: 456
Tamaño del conjunto de prueba(original): 404


In [11]:
#total 570
#train 456
#test (114) pero tengo 404 
#deberia ser 570 - 456

In [12]:
#nuevo intneto

In [13]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
# Cargar datos
train_df = pd.read_csv('train_without_clauses_including_adverbs.csv')
test_df = pd.read_csv('../Step_1/data.csv')


In [15]:
# Convertir texto a listas
train_texts = train_df['text'].astype(str).tolist()
test_texts = test_df['text'].astype(str).tolist()


In [16]:
# Tokenización y comparación Jaccard
def jaccard_similarity(text1, text2):
    set1, set2 = set(word_tokenize(text1.lower())), set(word_tokenize(text2.lower()))
    return len(set1 & set2) / len(set1 | set2) if set1 | set2 else 0


In [19]:
# Vectorización con TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(train_texts + test_texts)
# Separar las matrices de train y test
train_matrix = tfidf_matrix[:len(train_texts)]
test_matrix = tfidf_matrix[len(train_texts):]

threshold = 0.4  # Ajusta este valor según la similitud deseada


In [20]:
similarities = cosine_similarity(test_matrix, train_matrix)

similar_indices = [i for i, row in enumerate(similarities) if max(row) > threshold]


In [21]:
# Eliminar frases similares del test
test_df = test_df.drop(similar_indices)
test_df.to_csv('filtered_test.csv', index=False)

In [22]:
print(f"Tamaño del conjunto de entrenamiento: {len(train_df)}")
print(f"Tamaño del conjunto de prueba(original): {len(test_df)}")

Tamaño del conjunto de entrenamiento: 456
Tamaño del conjunto de prueba(original): 430


In [ ]:

        print(token.text)